In [1]:
import json
import numpy
from collections import OrderedDict
import sys

def load_hardware_trace_json(file):
    """
    Reads a timeline.json file output by Tensorflow/libcupti and returns and OrderedDict object
    :param file: .json file.
    :return: OrderedDict
    """
    with open(file, mode='r') as f:

        def _as_ordered_dict(val):
            return OrderedDict(val)

        def _as_list(val):
            return list(val)

        output = json.load(f, object_hook=_as_ordered_dict, object_pairs_hook=_as_ordered_dict)
        dic = OrderedDict(output)

    return dic

def get_all_ops(trace_dic):
    """
    Params: 
    trace_dic: collections.OrderedDict of traceEvent
    Return: list of dictionaries of all ops.
    """
    try:
        traceEvents = trace_dic['traceEvents']
    except KeyError:
        print('Not valid GPU trace dict object.')
        sys.exit()
    all_ops = []
    for trace in traceEvents:
        try:
            if trace['cat'] == 'Op':
                all_ops.append(trace)
        except KeyError:
            pass
    return all_ops

def get_stream_all(trace_dic):
    """
    Params: 
    trace_dic: collections.OrderedDict of traceEvent
    Return: pid of GPU/stream:all, (stream, pid) dictionary
    """
    try:
        traceEvents = trace_dic['traceEvents']
    except KeyError:
        print('Not valid GPU trace dict object.')
        sys.exit()
    all_procs = []
    for trace in traceEvents:
        try:
            if trace['name'] == 'process_name':
                all_procs.append((trace['args']['name'], trace['pid']))
        except KeyError:
            pass
    dic_procs = dict(all_procs)
    pid = dic_procs['/device:GPU:0/stream:all Compute']
    return dic_procs, pid

def get_unique_ops_names(all_ops):
    '''
    Find unique op names.
    Params: 
    all_ops: list, of dictionary of all operations.
    Return: list of unique op names.
    '''
    return set(op['name'] for op in all_ops)

def get_wall_duration(op_names, all_ops, pid_list=[11, 7, 13, 15, 9]):
    '''
    Calculates wall duration for each op in op_names.
    Params:
    op_names: list (str), names of ops of interest.
    pid_list: list (str), names of pid to include.
    all_ops: output of get_all_ops().
    Return:
    total wall duration, dict['op'] = wall duration.
    '''
   #1. Construct dictionary of op with name matching op_names 
    ops_dic=OrderedDict()
    for name in op_names:
        ops = []
        for op in all_ops:
            if op['name'] == name:
                ops.append(op)
        ops_dic[name] = ops
    
    #2. get duration for each op
    op_dict = OrderedDict()
    total_dur = 0
    for op_name in op_names:
        op_dur = 0
        for itm in ops_dic[op_name]:
            if itm['pid'] in pid_list:
                op_dur += itm['dur']
        op_dict[op_name] = op_dur*1e-3 # convert from us to ms
        total_dur += op_dur*1e-3
        
    print('Total Wall Duration (ms): %4.3f\n' % total_dur)
    sorted_dur = sorted(op_dict.items(), key=lambda x: x[1])[::-1]
    print('OPS with wall duration > 5 ms:')
    for itm in sorted_dur:
        if itm[1] > 5:
            if itm[0] == 'unknown':
                name = 'unknown (nccl AllReduceKernel_sum_)'
            else:
                name = itm[0]
            print('%s : %3.3f ms' %(name,itm[1]))
    return total_dur, op_dict

In [2]:
# Load timeline and get all ops information
dic = load_hardware_trace_json('resnet_50_heavy/timeline.ctf.3.json')
all_ops = get_all_ops(dic)
# Find Names of unique Ops
unique_op_names = get_unique_ops_names(all_ops)
print(unique_op_names)

{'Mul', 'Identity', 'PreventGradient', 'Conv2DBackpropFilter', 'MEMCPYHtoD', 'Pow', 'LessEqual', 'Tile', 'Less', 'AvgPool', 'Switch', 'Conv2D', 'Merge', 'ExpandDims', 'L2Loss', 'SparseSoftmaxCrossEntropyWithLogits', 'AddN', 'Maximum', 'unknown', '_ParallelConcatStart', 'Sub', 'MaxPoolGrad', 'ArgMax', 'Const', 'DecodeRaw', 'Sqrt', 'MEMCPYDtoH', 'Equal', 'Mean', 'FusedBatchNormGradV2', '_ParallelConcatUpdate', 'BiasAdd', 'BiasAddGrad', 'Conv2DBackpropInput', 'Cast', 'Add', 'Floor', 'Square', 'AssignSub', 'Split', 'AvgPoolGrad', 'StridedSlice', 'RecordInput', 'Transpose', 'AssignAdd', 'HorovodAllreduce', 'ConcatV2', 'MEMCPYDtoD', 'MatMul', 'ApplyMomentum', 'Reshape', 'FusedBatchNormV2', 'NoOp', 'Unstage', 'MaxPool', 'ParseSingleExample', 'ReluGrad', 'RealDiv', 'Relu', 'Stage', 'VariableV2'}


In [3]:
proc_dic, stream_all_pid = get_stream_all(trace_dic=dic)
proc_dic

{'/device:GPU:0/memcpy Compute': 9,
 '/device:GPU:0/memcpy Tensors': 10,
 '/device:GPU:0/stream:13 Compute': 17,
 '/device:GPU:0/stream:13 Tensors': 18,
 '/device:GPU:0/stream:14 Compute': 5,
 '/device:GPU:0/stream:14 Tensors': 6,
 '/device:GPU:0/stream:15 Compute': 1,
 '/device:GPU:0/stream:15 Tensors': 2,
 '/device:GPU:0/stream:17 Compute': 7,
 '/device:GPU:0/stream:17 Tensors': 8,
 '/device:GPU:0/stream:18 Compute': 15,
 '/device:GPU:0/stream:18 Tensors': 16,
 '/device:GPU:0/stream:26 Compute': 3,
 '/device:GPU:0/stream:26 Tensors': 4,
 '/device:GPU:0/stream:all Compute': 11,
 '/device:GPU:0/stream:all Tensors': 12,
 '/job:localhost/replica:0/task:0/device:CPU:0 Compute': 13,
 '/job:localhost/replica:0/task:0/device:CPU:0 Tensors': 14,
 '/job:localhost/replica:0/task:0/device:GPU:0 Compute': 19,
 '/job:localhost/replica:0/task:0/device:GPU:0 Tensors': 20,
 'Allocators': 0}

In [4]:
# Calculate wall duration of all ops
total_dur, dur_dic_all = get_wall_duration(unique_op_names, all_ops, pid_list=[stream_all_pid])

Total Wall Duration (ms): 2250.133

OPS with wall duration > 5 ms:
Conv2D : 1160.922 ms
Conv2DBackpropFilter : 569.469 ms
unknown (nccl AllReduceKernel_sum_) : 297.506 ms
Conv2DBackpropInput : 90.028 ms
FusedBatchNormGradV2 : 28.982 ms
ReluGrad : 24.394 ms
MaxPoolGrad : 16.852 ms
FusedBatchNormV2 : 14.570 ms
Relu : 11.501 ms
AddN : 7.347 ms
Add : 6.795 ms
BiasAdd : 6.178 ms
